<a href="https://colab.research.google.com/github/MHS-Group21/GroupProject/blob/master/MLM_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# THIS IS DIFFERENT MACHINE LEARNING MODELS WITH ONE-HOT ENCODING & TEAM FORMS

---



1.   The data prediction doesn't seem to be improving based on the team forms so i need to figure out a way to see if it has an effect on the final prediction rate and also maybe instead of just seeing the previous 5 games results, try find out what other metrics i should use 
2.   Introduce label encoding on a different list to do feature selection
3. Try different ML models to see if it makes a huge difference.
4. Test using cross_val_score 



# Imports and Setup


In [47]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# matches=pd.read_csv('All Data Final.csv')
matches=pd.read_csv('All Data Final ABV.csv')
matches.head()

,season,date,day_games,team1,team2,venue,city,country,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,umpire1,umpire2
0,2017,05/04/2017,0,SRH,RCB,Rajiv Gandhi International Cricket Stadium,Hyderabad,India,RCB,field,normal,0,SRH,35,0,Yuvraj Singh,AY Dandekar,NJ Llong
1,2017,06/04/2017,0,MI,RPS,Maharashtra Cricket Association Stadium,Pune,India,RPS,field,normal,0,RPS,0,7,SPD Smith,A Nand Kishore,S Ravi
2,2017,07/04/2017,0,GL,KKR,Saurashtra Cricket Association Stadium,Rajkot,India,KKR,field,normal,0,KKR,0,10,CA Lynn,Nitin Menon,CK Nandan
3,2017,08/04/2017,1,RPS,PBKS,Holkar Cricket Stadium,Indore,India,PBKS,field,normal,0,PBKS,0,6,GJ Maxwell,AK Chaudhary,C Shamshuddin
4,2017,08/04/2017,0,RCB,DC,M Chinnaswamy Stadium,Bangalore,India,RCB,bat,normal,0,RCB,15,0,KM Jadhav,NaN,NaN


In [ ]:
# matches=pd.read_csv('All Data Final.csv')
# print(*(sorted(matches['venue'].unique())), sep='\n')
# matches['venue'].nunique()

# Creating new features

In [48]:
#outcome variable as a probability of team1 winning
matches.loc[matches['winner']==matches['team1'],'team1_win']=1
matches.loc[matches['winner']!=matches['team1'],'team1_win']=0

matches.loc[matches['toss_winner']==matches['team1'],'team1_toss_win']=1
matches.loc[matches['toss_winner']!=matches['team1'],'team1_toss_win']=0

matches['team1_bat']=0
matches.loc[(matches['team1_toss_win']==1) & (matches['toss_decision']=='bat'),'team1_bat']=1

# prediction_df=matches[['team1','team2','team1_toss_win','team1_bat','team1_win','venue','day_games']]
# prediction_df.head()
matches.head()

,season,date,day_games,team1,team2,venue,city,country,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,umpire1,umpire2,team1_win,team1_toss_win,team1_bat
0,2017,05/04/2017,0,SRH,RCB,Rajiv Gandhi International Cricket Stadium,Hyderabad,India,RCB,field,normal,0,SRH,35,0,Yuvraj Singh,AY Dandekar,NJ Llong,1.0,0.0,0
1,2017,06/04/2017,0,MI,RPS,Maharashtra Cricket Association Stadium,Pune,India,RPS,field,normal,0,RPS,0,7,SPD Smith,A Nand Kishore,S Ravi,0.0,0.0,0
2,2017,07/04/2017,0,GL,KKR,Saurashtra Cricket Association Stadium,Rajkot,India,KKR,field,normal,0,KKR,0,10,CA Lynn,Nitin Menon,CK Nandan,0.0,0.0,0
3,2017,08/04/2017,1,RPS,PBKS,Holkar Cricket Stadium,Indore,India,PBKS,field,normal,0,PBKS,0,6,GJ Maxwell,AK Chaudhary,C Shamshuddin,0.0,0.0,0
4,2017,08/04/2017,0,RCB,DC,M Chinnaswamy Stadium,Bangalore,India,RCB,bat,normal,0,RCB,15,0,KM Jadhav,NaN,NaN,1.0,1.0,1


# Venue Link

In [49]:
# Making team1_home and team2_home column for all matches
matches['team1_home']=0
matches['team2_home']=0

In [ ]:
# # For Chennai Super Kings as Team 1 
# matches.loc[(matches['team1']=='Chennai Super Kings') & (matches['venue']=='MA Chidambaram Stadium'),'team1_home']=1
# matches.loc[(matches['team1']=='Chennai Super Kings') & (matches['venue']=='JSCA International Stadium Complex') & (matches['season'] == 2014),'team1_home']=1
# matches.loc[(matches['team1']=='Chennai Super Kings') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'] == 2018),'team1_home']=1

# # For Chennai Super Kings as Team 2 
# matches.loc[(matches['team2']=='Chennai Super Kings') & (matches['venue']=='MA Chidambaram Stadium'),'team2_home']=1
# matches.loc[(matches['team2']=='Chennai Super Kings') & (matches['venue']=='JSCA International Stadium Complex') & (matches['season'] == 2014),'team2_home']=1
# matches.loc[(matches['team2']=='Chennai Super Kings') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'] == 2018),'team2_home']=1

# # ============================ #

# # For Deccan Chargeres as Team 1 
# matches.loc[(matches['team1']=='Deccan Chargers') & (matches['venue']=='Rajiv Gandhi International Cricket Stadium') & (matches['season'].isin([2008, 2009, 2010, 2011, 2012])),'team1_home']=1
# matches.loc[(matches['team1']=='Deccan Chargers') & (matches['venue']=='Barabati Stadium') & (matches['season'].isin([2010, 2011, 2012])),'team1_home']=1
# matches.loc[(matches['team1']=='Deccan Chargers') & (matches['venue']=='Vidarbha Cricket Stadium') & (matches['season'] == 2010),'team1_home']=1
# matches.loc[(matches['team1']=='Deccan Chargers') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season'] == 2012),'team1_home']=1
# matches.loc[(matches['team1']=='Deccan Chargers') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season'] == 2010),'team1_home']=1

# # For Deccan Chargeres as Team 2
# matches.loc[(matches['team2']=='Deccan Chargers') & (matches['venue']=='Rajiv Gandhi International Cricket Stadium') & (matches['season'].isin([2008, 2009, 2010, 2011, 2012])),'team2_home']=1
# matches.loc[(matches['team2']=='Deccan Chargers') & (matches['venue']=='Barabati Stadium') & (matches['season'].isin([2010, 2011, 2012])),'team2_home']=1
# matches.loc[(matches['team2']=='Deccan Chargers') & (matches['venue']=='Vidarbha Cricket Stadium') & (matches['season'] == 2010),'team2_home']=1
# matches.loc[(matches['team2']=='Deccan Chargers') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season'] == 2012),'team2_home']=1
# matches.loc[(matches['team2']=='Deccan Chargers') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season'] == 2010),'team2_home']=1

# # ============================ #

# # For Delhi Capitals as Team 1
# matches.loc[(matches['team1']=='Delhi Capitals') & (matches['venue']=='Arun Jaitley Stadium'),'team1_home']=1
# matches.loc[(matches['team1']=='Delhi Capitals') & (matches['venue']=='Shaheed Veer Narayan Singh International Stadium') & (matches['season'].isin([2013, 2014, 2015, 2016])),'team1_home']=1

# # For Delhi Capitals as Team 2
# matches.loc[(matches['team2']=='Delhi Capitals') & (matches['venue']=='Arun Jaitley Stadium'),'team2_home']=1
# matches.loc[(matches['team2']=='Delhi Capitals') & (matches['venue']=='Shaheed Veer Narayan Singh International Stadium') & (matches['season'].isin([2013, 2014, 2015, 2016])),'team2_home']=1

# # ============================ #

# # For Gujarat Lions as Team 1
# matches.loc[(matches['team1']=='Gujarat Lions') & (matches['venue']=='Saurashtra Cricket Association Stadium') & (matches['season'].isin([2016, 2017])),'team1_home']=1
# matches.loc[(matches['team1']=='Gujarat Lions') & (matches['venue']=='Green Park') & (matches['season'].isin([2016, 2017])),'team1_home']=1

# # For Gujarat Lions as Team 2
# matches.loc[(matches['team2']=='Gujarat Lions') & (matches['venue']=='Saurashtra Cricket Association Stadium') & (matches['season'].isin([2016, 2017])),'team2_home']=1
# matches.loc[(matches['team2']=='Gujarat Lions') & (matches['venue']=='Green Park') & (matches['season'].isin([2016, 2017])),'team2_home']=1

# # ============================ #

# # For Punjab Kings as Team 1
# matches.loc[(matches['team1']=='Punjab Kings') & (matches['venue']=='Punjab Cricket Association Cricket Stadium'), 'team1_home']=1
# matches.loc[(matches['team1']=='Punjab Kings') & (matches['venue']=='Himachal Pradesh Cricket Association Stadium') & (matches['season'].isin([2010,2011,2012,2013])),'team1_home']=1
# matches.loc[(matches['team1']=='Punjab Kings') & (matches['venue']=='Barabati Stadium') & (matches['season']==2014), 'team1_home']=1
# matches.loc[(matches['team1']=='Punjab Kings') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season']==2015), 'team1_home']=1
# matches.loc[(matches['team1']=='Punjab Kings') & (matches['venue']=='Holkar Cricket Stadium') & (matches['season'].isin([2017,2018])),'team1_home']=1

# # For Punjab Kings as Team 2
# matches.loc[(matches['team2']=='Punjab Kings') & (matches['venue']=='Punjab Cricket Association Cricket Stadium'), 'team2_home']=1
# matches.loc[(matches['team2']=='Punjab Kings') & (matches['venue']=='Himachal Pradesh Cricket Association Stadium') & (matches['season'].isin([2010,2011,2012,2013])),'team2_home']=1
# matches.loc[(matches['team2']=='Punjab Kings') & (matches['venue']=='Barabati Stadium') & (matches['season']==2014), 'team2_home']=1
# matches.loc[(matches['team2']=='Punjab Kings') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season']==2015), 'team2_home']=1
# matches.loc[(matches['team2']=='Punjab Kings') & (matches['venue']=='Holkar Cricket Stadium') & (matches['season'].isin([2017,2018])),'team2_home']=1

# # ============================ #

# # For Kochi Tuskers Kerala as Team 1
# matches.loc[(matches['team1']=='Kochi Tuskers Kerala') & (matches['venue']=='Jawaharlal Nehru Stadium') & (matches['season']==2011), 'team1_home']=1
# matches.loc[(matches['team1']=='Kochi Tuskers Kerala') & (matches['venue']=='Holkar Cricket Stadium') & (matches['season']==2011), 'team1_home']=1

# # For Kochi Tuskers Kerala as Team 2
# matches.loc[(matches['team2']=='Kochi Tuskers Kerala') & (matches['venue']=='Jawaharlal Nehru Stadium') & (matches['season']==2011), 'team2_home']=1
# matches.loc[(matches['team2']=='Kochi Tuskers Kerala') & (matches['venue']=='Holkar Cricket Stadium') & (matches['season']==2011), 'team2_home']=1

# # ============================ #

# # For Kolkata Knight Riders as Team 1
# matches.loc[(matches['team1']=='Kolkata Knight Riders') & (matches['venue']=='Eden Gardens'), 'team1_home']=1
# matches.loc[(matches['team1']=='Kolkata Knight Riders') & (matches['venue']=='JSCA International Stadium Complex') & (matches['season']==2013), 'team1_home']=1

# # For Kolkata Knight Riders as Team 2
# matches.loc[(matches['team2']=='Kolkata Knight Riders') & (matches['venue']=='Eden Gardens'), 'team2_home']=1
# matches.loc[(matches['team2']=='Kolkata Knight Riders') & (matches['venue']=='JSCA International Stadium Complex') & (matches['season']==2013), 'team2_home']=1

# # ============================ #

# # For Mumbai Indians as Team 1
# matches.loc[(matches['team1']=='Mumbai Indians') & (matches['venue']=='Wankhede Stadium'), 'team1_home']=1
# matches.loc[(matches['team1']=='Mumbai Indians') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season']==2008), 'team1_home']=1
# matches.loc[(matches['team1']=='Mumbai Indians') & (matches['venue']=='Brabourne Stadium') & (matches['season']==2010), 'team1_home']=1
# matches.loc[(matches['team1']=='Mumbai Indians') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2016), 'team1_home']=1

# # For Mumbai Indians as Team 2
# matches.loc[(matches['team2']=='Mumbai Indians') & (matches['venue']=='Wankhede Stadium'), 'team2_home']=1
# matches.loc[(matches['team2']=='Mumbai Indians') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season']==2008), 'team2_home']=1
# matches.loc[(matches['team2']=='Mumbai Indians') & (matches['venue']=='Brabourne Stadium') & (matches['season']==2010), 'team2_home']=1
# matches.loc[(matches['team2']=='Mumbai Indians') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2016), 'team2_home']=1

# # ============================ #

# # For Pune Warriors India as Team 1
# matches.loc[(matches['team1']=='Pune Warriors India') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season']==2011), 'team1_home']=1
# matches.loc[(matches['team1']=='Pune Warriors India') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'].isin([2012,2013])), 'team1_home']=1

# # For Pune Warriors India as Team 2
# matches.loc[(matches['team2']=='Pune Warriors India') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season']==2011), 'team2_home']=1
# matches.loc[(matches['team2']=='Pune Warriors India') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'].isin([2012,2013])), 'team2_home']=1

# # ============================ #

# # For Rajasthan Royals as Team 1
# matches.loc[(matches['team1']=='Rajasthan Royals') & (matches['venue']=='Sawai Mansingh Stadium'), 'team1_home']=1
# matches.loc[(matches['team1']=='Rajasthan Royals') & (matches['venue']=='Narendra Modi Stadium') & (matches['season'].isin([2010,2014,2015])), 'team1_home']=1
# matches.loc[(matches['team1']=='Rajasthan Royals') & (matches['venue']=='Brabourne Stadium') & (matches['season']==2015), 'team1_home']=1

# # For Rajasthan Royals as Team 2
# matches.loc[(matches['team2']=='Rajasthan Royals') & (matches['venue']=='Sawai Mansingh Stadium'), 'team2_home']=1
# matches.loc[(matches['team2']=='Rajasthan Royals') & (matches['venue']=='Narendra Modi Stadium') & (matches['season'].isin([2010,2014,2015])), 'team2_home']=1
# matches.loc[(matches['team2']=='Rajasthan Royals') & (matches['venue']=='Brabourne Stadium') & (matches['season']==2015), 'team2_home']=1

# # ============================ #

# # For Rising Pune Supergiants as Team 1
# matches.loc[(matches['team1']=='Rising Pune Supergiants') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'].isin([2016,2017])), 'team1_home']=1
# matches.loc[(matches['team1']=='Rising Pune Supergiants') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2016), 'team1_home']=1

# # For Rising Pune Supergiants as Team 2
# matches.loc[(matches['team2']=='Rising Pune Supergiants') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'].isin([2016,2017])), 'team2_home']=1
# matches.loc[(matches['team2']=='Rising Pune Supergiants') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2016), 'team2_home']=1

# # ============================ #

# # For Royal Challengers Bangalore as Team 1
# matches.loc[(matches['team1']=='Royal Challengers Bangalore') & (matches['venue']=='M Chinnaswamy Stadium'), 'team1_home']=1

# # For Royal Challengers Bangalore as Team 2
# matches.loc[(matches['team2']=='Royal Challengers Bangalore') & (matches['venue']=='M Chinnaswamy Stadium'), 'team2_home']=1

# # ============================ #

# # For Sunrisers Hyderabad as Team 1
# matches.loc[(matches['team1']=='Sunrisers Hyderabad') & (matches['venue']=='Rajiv Gandhi International Cricket Stadium'), 'team1_home']=1
# matches.loc[(matches['team1']=='Sunrisers Hyderabad') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2015), 'team1_home']=1

# # For Sunrisers Hyderabad as Team 2
# matches.loc[(matches['team2']=='Sunrisers Hyderabad') & (matches['venue']=='Rajiv Gandhi International Cricket Stadium'), 'team2_home']=1
# matches.loc[(matches['team2']=='Sunrisers Hyderabad') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2015), 'team2_home']=1


In [50]:
# For CSK as Team 1 
matches.loc[(matches['team1']=='CSK') & (matches['venue']=='MA Chidambaram Stadium'),'team1_home']=1
matches.loc[(matches['team1']=='CSK') & (matches['venue']=='JSCA International Stadium Complex') & (matches['season'] == 2014),'team1_home']=1
matches.loc[(matches['team1']=='CSK') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'] == 2018),'team1_home']=1

# For CSK as Team 2 
matches.loc[(matches['team2']=='CSK') & (matches['venue']=='MA Chidambaram Stadium'),'team2_home']=1
matches.loc[(matches['team2']=='CSK') & (matches['venue']=='JSCA International Stadium Complex') & (matches['season'] == 2014),'team2_home']=1
matches.loc[(matches['team2']=='CSK') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'] == 2018),'team2_home']=1

# ============================ #

# For Deccan Chargers as Team 1 
matches.loc[(matches['team1']=='DEC') & (matches['venue']=='Rajiv Gandhi International Cricket Stadium') & (matches['season'].isin([2008, 2009, 2010, 2011, 2012])),'team1_home']=1
matches.loc[(matches['team1']=='DEC') & (matches['venue']=='Barabati Stadium') & (matches['season'].isin([2010, 2011, 2012])),'team1_home']=1
matches.loc[(matches['team1']=='DEC') & (matches['venue']=='Vidarbha Cricket Stadium') & (matches['season'] == 2010),'team1_home']=1
matches.loc[(matches['team1']=='DEC') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season'] == 2012),'team1_home']=1
matches.loc[(matches['team1']=='DEC') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season'] == 2010),'team1_home']=1

# For Deccan Chargers as Team 2
matches.loc[(matches['team2']=='DEC') & (matches['venue']=='Rajiv Gandhi International Cricket Stadium') & (matches['season'].isin([2008, 2009, 2010, 2011, 2012])),'team2_home']=1
matches.loc[(matches['team2']=='DEC') & (matches['venue']=='Barabati Stadium') & (matches['season'].isin([2010, 2011, 2012])),'team2_home']=1
matches.loc[(matches['team2']=='DEC') & (matches['venue']=='Vidarbha Cricket Stadium') & (matches['season'] == 2010),'team2_home']=1
matches.loc[(matches['team2']=='DEC') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season'] == 2012),'team2_home']=1
matches.loc[(matches['team2']=='DEC') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season'] == 2010),'team2_home']=1

# ============================ #

# For DC as Team 1
matches.loc[(matches['team1']=='DC') & (matches['venue']=='Arun Jaitley Stadium'),'team1_home']=1
matches.loc[(matches['team1']=='DC') & (matches['venue']=='Shaheed Veer Narayan Singh International Stadium') & (matches['season'].isin([2013, 2014, 2015, 2016])),'team1_home']=1

# For DC as Team 2
matches.loc[(matches['team2']=='DC') & (matches['venue']=='Arun Jaitley Stadium'),'team2_home']=1
matches.loc[(matches['team2']=='DC') & (matches['venue']=='Shaheed Veer Narayan Singh International Stadium') & (matches['season'].isin([2013, 2014, 2015, 2016])),'team2_home']=1

# ============================ #

# For GL as Team 1
matches.loc[(matches['team1']=='GL') & (matches['venue']=='Saurashtra Cricket Association Stadium') & (matches['season'].isin([2016, 2017])),'team1_home']=1
matches.loc[(matches['team1']=='GL') & (matches['venue']=='Green Park') & (matches['season'].isin([2016, 2017])),'team1_home']=1

# For GL as Team 2
matches.loc[(matches['team2']=='GL') & (matches['venue']=='Saurashtra Cricket Association Stadium') & (matches['season'].isin([2016, 2017])),'team2_home']=1
matches.loc[(matches['team2']=='GL') & (matches['venue']=='Green Park') & (matches['season'].isin([2016, 2017])),'team2_home']=1

# ============================ #

# For PBKS as Team 1
matches.loc[(matches['team1']=='PBKS') & (matches['venue']=='Punjab Cricket Association Cricket Stadium'), 'team1_home']=1
matches.loc[(matches['team1']=='PBKS') & (matches['venue']=='Himachal Pradesh Cricket Association Stadium') & (matches['season'].isin([2010,2011,2012,2013])),'team1_home']=1
matches.loc[(matches['team1']=='PBKS') & (matches['venue']=='Barabati Stadium') & (matches['season']==2014), 'team1_home']=1
matches.loc[(matches['team1']=='PBKS') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season']==2015), 'team1_home']=1
matches.loc[(matches['team1']=='PBKS') & (matches['venue']=='Holkar Cricket Stadium') & (matches['season'].isin([2017,2018])),'team1_home']=1

# For PBKS as Team 2
matches.loc[(matches['team2']=='PBKS') & (matches['venue']=='Punjab Cricket Association Cricket Stadium'), 'team2_home']=1
matches.loc[(matches['team2']=='PBKS') & (matches['venue']=='Himachal Pradesh Cricket Association Stadium') & (matches['season'].isin([2010,2011,2012,2013])),'team2_home']=1
matches.loc[(matches['team2']=='PBKS') & (matches['venue']=='Barabati Stadium') & (matches['season']==2014), 'team2_home']=1
matches.loc[(matches['team2']=='PBKS') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season']==2015), 'team2_home']=1
matches.loc[(matches['team2']=='PBKS') & (matches['venue']=='Holkar Cricket Stadium') & (matches['season'].isin([2017,2018])),'team2_home']=1

# ============================ #

# For KTK as Team 1
matches.loc[(matches['team1']=='KTK') & (matches['venue']=='Jawaharlal Nehru Stadium') & (matches['season']==2011), 'team1_home']=1
matches.loc[(matches['team1']=='KTK') & (matches['venue']=='Holkar Cricket Stadium') & (matches['season']==2011), 'team1_home']=1

# For KTK as Team 2
matches.loc[(matches['team2']=='KTK') & (matches['venue']=='Jawaharlal Nehru Stadium') & (matches['season']==2011), 'team2_home']=1
matches.loc[(matches['team2']=='KTK') & (matches['venue']=='Holkar Cricket Stadium') & (matches['season']==2011), 'team2_home']=1

# ============================ #

# For KKR as Team 1
matches.loc[(matches['team1']=='KKR') & (matches['venue']=='Eden Gardens'), 'team1_home']=1
matches.loc[(matches['team1']=='KKR') & (matches['venue']=='JSCA International Stadium Complex') & (matches['season']==2013), 'team1_home']=1

# For KKR as Team 2
matches.loc[(matches['team2']=='KKR') & (matches['venue']=='Eden Gardens'), 'team2_home']=1
matches.loc[(matches['team2']=='KKR') & (matches['venue']=='JSCA International Stadium Complex') & (matches['season']==2013), 'team2_home']=1

# ============================ #

# For MI as Team 1
matches.loc[(matches['team1']=='MI') & (matches['venue']=='Wankhede Stadium'), 'team1_home']=1
matches.loc[(matches['team1']=='MI') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season']==2008), 'team1_home']=1
matches.loc[(matches['team1']=='MI') & (matches['venue']=='Brabourne Stadium') & (matches['season']==2010), 'team1_home']=1
matches.loc[(matches['team1']=='MI') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2016), 'team1_home']=1

# For MI as Team 2
matches.loc[(matches['team2']=='MI') & (matches['venue']=='Wankhede Stadium'), 'team2_home']=1
matches.loc[(matches['team2']=='MI') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season']==2008), 'team2_home']=1
matches.loc[(matches['team2']=='MI') & (matches['venue']=='Brabourne Stadium') & (matches['season']==2010), 'team2_home']=1
matches.loc[(matches['team2']=='MI') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2016), 'team2_home']=1

# ============================ #

# For PWI as Team 1
matches.loc[(matches['team1']=='PWI') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season']==2011), 'team1_home']=1
matches.loc[(matches['team1']=='PWI') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'].isin([2012,2013])), 'team1_home']=1

# For PWI as Team 2
matches.loc[(matches['team2']=='PWI') & (matches['venue']=='Dr. D.Y. Patil Sports Academy') & (matches['season']==2011), 'team2_home']=1
matches.loc[(matches['team2']=='PWI') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'].isin([2012,2013])), 'team2_home']=1

# ============================ #

# For RR as Team 1
matches.loc[(matches['team1']=='RR') & (matches['venue']=='Sawai Mansingh Stadium'), 'team1_home']=1
matches.loc[(matches['team1']=='RR') & (matches['venue']=='Narendra Modi Stadium') & (matches['season'].isin([2010,2014,2015])), 'team1_home']=1
matches.loc[(matches['team1']=='RR') & (matches['venue']=='Brabourne Stadium') & (matches['season']==2015), 'team1_home']=1

# For RR as Team 2
matches.loc[(matches['team2']=='RR') & (matches['venue']=='Sawai Mansingh Stadium'), 'team2_home']=1
matches.loc[(matches['team2']=='RR') & (matches['venue']=='Narendra Modi Stadium') & (matches['season'].isin([2010,2014,2015])), 'team2_home']=1
matches.loc[(matches['team2']=='RR') & (matches['venue']=='Brabourne Stadium') & (matches['season']==2015), 'team2_home']=1

# ============================ #

# For RPS as Team 1
matches.loc[(matches['team1']=='RPS') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'].isin([2016,2017])), 'team1_home']=1
matches.loc[(matches['team1']=='RPS') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2016), 'team1_home']=1

# For RPS as Team 2
matches.loc[(matches['team2']=='RPS') & (matches['venue']=='Maharashtra Cricket Association Stadium') & (matches['season'].isin([2016,2017])), 'team2_home']=1
matches.loc[(matches['team2']=='RPS') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2016), 'team2_home']=1

# ============================ #

# For RCB as Team 1
matches.loc[(matches['team1']=='RCB') & (matches['venue']=='M Chinnaswamy Stadium'), 'team1_home']=1

# For RCB as Team 2
matches.loc[(matches['team2']=='RCB') & (matches['venue']=='M Chinnaswamy Stadium'), 'team2_home']=1

# ============================ #

# For SRH as Team 1
matches.loc[(matches['team1']=='SRH') & (matches['venue']=='Rajiv Gandhi International Cricket Stadium'), 'team1_home']=1
matches.loc[(matches['team1']=='SRH') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2015), 'team1_home']=1

# For SRH as Team 2
matches.loc[(matches['team2']=='SRH') & (matches['venue']=='Rajiv Gandhi International Cricket Stadium'), 'team2_home']=1
matches.loc[(matches['team2']=='SRH') & (matches['venue']=='Dr. Y.S.R. ACA VDCA Cricket Stadium') & (matches['season']==2015), 'team2_home']=1

In [51]:
# Making team1_home and team2_home 0 for games played outside India
matches.loc[matches['country']!='India','team1_home']=0
matches.loc[matches['country']!='India','team2_home']=0

# Making team1_home and team2_home 0 for games played in India in 2021
matches.loc[(matches['country']=='India') & (matches['season'] == 2021),'team1_home']=0
matches.loc[(matches['country']=='India') & (matches['season'] == 2021),'team2_home']=0

matches

,season,date,day_games,team1,team2,venue,city,country,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,umpire1,umpire2,team1_win,team1_toss_win,team1_bat,team1_home,team2_home
0,2017,05/04/2017,0,SRH,RCB,Rajiv Gandhi International Cricket Stadium,Hyderabad,India,RCB,field,normal,0,SRH,35,0,Yuvraj Singh,AY Dandekar,NJ Llong,1.0,0.0,0,1,0
1,2017,06/04/2017,0,MI,RPS,Maharashtra Cricket Association Stadium,Pune,India,RPS,field,normal,0,RPS,0,7,SPD Smith,A Nand Kishore,S Ravi,0.0,0.0,0,0,1
2,2017,07/04/2017,0,GL,KKR,Saurashtra Cricket Association Stadium,Rajkot,India,KKR,field,normal,0,KKR,0,10,CA Lynn,Nitin Menon,CK Nandan,0.0,0.0,0,1,0
3,2017,08/04/2017,1,RPS,PBKS,Holkar Cricket Stadium,Indore,India,PBKS,field,normal,0,PBKS,0,6,GJ Maxwell,AK Chaudhary,C Shamshuddin,0.0,0.0,0,0,1
4,2017,08/04/2017,0,RCB,DC,M Chinnaswamy Stadium,Bangalore,India,RCB,bat,normal,0,RCB,15,0,KM Jadhav,NaN,NaN,1.0,1.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,2021,08/10/2021,0,RCB,DC,Dubai International Cricket Stadium,Dubai,UAE,RCB,Field,normal,0,RCB,0,7,NaN,NaN,NaN,1.0,1.0,0,0,0
868,2021,10/10/2021,0,DC,CSK,Dubai International Cricket Stadium,Dubai,UAE,CSK,Field,normal,0,CSK,0,4,NaN,NaN,NaN,0.0,0.0,0,0,0
869,2021,11/10/2021,0,RCB,KKR,Sharjah Cricket Stadium,Sharjah,UAE,RCB,Bat,normal,0,KKR,0,4,NaN,NaN,NaN,0.0,1.0,0,0,0
870,2021,13/10/2021,0,DC,KKR,Sharjah Cricket Stadium,Sharjah,UAE,KKR,Field,normal,0,KKR,0,3,NaN,NaN,NaN,0.0,0.0,0,0,0


In [52]:
# Write test cases to see if the code above did the work correctly.
# See if there are games with 1 for both team1_home and team2_home
# See if there are games with 0 for both team1_home and team2_home for games played before 2020

# matches.loc[(matches['team1_home']==1) & (matches['team2_home']==1)]
matches.loc[(matches['team1_home']==0) & (matches['team2_home']==0) & (matches['season'] <2020) & (matches['country'] == 'India')]


,season,date,day_games,team1,team2,venue,city,country,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,umpire1,umpire2,team1_win,team1_toss_win,team1_bat,team1_home,team2_home
56,2017,17/05/2017,0,SRH,KKR,M Chinnaswamy Stadium,Bangalore,India,KKR,field,normal,1,KKR,0,7,NM Coulter-Nile,AK Chaudhary,Nitin Menon,0.0,0.0,0,0,0
57,2017,19/05/2017,0,KKR,MI,M Chinnaswamy Stadium,Bangalore,India,MI,field,normal,0,MI,0,6,KV Sharma,NJ Llong,Nitin Menon,0.0,0.0,0,0,0
58,2017,21/05/2017,0,MI,RPS,Rajiv Gandhi International Cricket Stadium,Hyderabad,India,MI,bat,normal,0,MI,1,0,KH Pandya,NJ Llong,S Ravi,1.0,1.0,1,0,0
71,2008,27/04/2008,0,MI,DEC,Dr DY Patil Sports Academy,Mumbai,India,DEC,field,normal,0,DEC,0,10,AC Gilchrist,Asad Rauf,SL Shastri,0.0,0.0,0,0,0
81,2008,04/05/2008,1,MI,DC,Dr DY Patil Sports Academy,Mumbai,India,DC,field,normal,0,MI,29,0,SM Pollock,IL Howell,RE Koertzen,1.0,0.0,0,0,0
85,2008,07/05/2008,0,RR,MI,Dr DY Patil Sports Academy,Mumbai,India,MI,field,normal,0,MI,0,7,A Nehra,DJ Harper,RE Koertzen,0.0,0.0,0,0,0
114,2008,30/05/2008,0,RR,DC,Wankhede Stadium,Mumbai,India,DC,field,normal,0,RR,105,0,SR Watson,BF Bowden,RE Koertzen,1.0,0.0,0,0,0
115,2008,31/05/2008,0,PBKS,CSK,Wankhede Stadium,Mumbai,India,PBKS,bat,normal,0,CSK,0,9,M Ntini,Asad Rauf,DJ Harper,0.0,1.0,1,0,0
116,2008,01/06/2008,0,CSK,RR,Dr DY Patil Sports Academy,Mumbai,India,RR,field,normal,0,RR,0,3,YK Pathan,BF Bowden,RE Koertzen,0.0,0.0,0,0,0
174,2010,12/03/2010,0,KKR,DEC,Dr DY Patil Sports Academy,Mumbai,India,DEC,field,normal,0,KKR,11,0,AD Mathews,RE Koertzen,RB Tiffin,1.0,0.0,0,0,0


# Previous Encounters

In [ ]:
# new columns created to show whether the Team1 won their last 5 games
matches['Home_1st_Last_Win'] = 0
matches['Home_2nd_Last_Win'] = 0
matches['Home_3rd_Last_Win'] = 0
matches['Home_4th_Last_Win'] = 0
matches['Home_5th_Last_Win'] = 0


# new columns created to show whether the Team2 won their last 5 games
matches['Away_1st_Last_Win'] = 0
matches['Away_2nd_Last_Win'] = 0
matches['Away_3rd_Last_Win'] = 0
matches['Away_4th_Last_Win'] = 0
matches['Away_5th_Last_Win'] = 0

# new columns created to show whether the Team1 won against the same Team2 in their previous 3 encounters
matches['Home_Last_Win_Against_Away'] = 0
matches['Home_2nd_Last_Win_Against_Away'] = 0
matches['Home_3rd_Last_Win_Against_Away'] = 0

matches.head()

In [ ]:
prediction_df=matches[['team1','team2','team1_toss_win','team1_bat','winner', 'team1_win','venue',
                       'Home_1st_Last_Win', 'Home_2nd_Last_Win', 'Home_3rd_Last_Win', 'Home_4th_Last_Win', 'Home_5th_Last_Win', 
                       'Away_1st_Last_Win', 'Away_2nd_Last_Win', 'Away_3rd_Last_Win', 'Away_4th_Last_Win', 'Away_5th_Last_Win', 
                       'Home_Last_Win_Against_Away', 'Home_2nd_Last_Win_Against_Away', 'Home_3rd_Last_Win_Against_Away']]
prediction = prediction_df

# **Add previous encounters and teams form** 

# Function to update the Team1's results (Won or Lost) of the last 5 games

In [ ]:
# def add_home_team_previous_form():
    
# Array to store the previous form of the Team1 
home_team_form = []

# The main loop that iterates through each row
for index, row in prediction.iterrows():    

    # Team1 of the current row
    current_home_team = row['team1'] 
    
    # The column indexes of the home and Team2s and the winner
    home_team_col = prediction.columns.get_loc('team1')
    away_team_col = prediction.columns.get_loc('team1')
    winner_col = prediction.columns.get_loc('winner')

    # The column indexes of the Team1's form (last 5 games) 
    home_1st_col = prediction.columns.get_loc('Home_1st_Last_Win')        
    home_2nd_col = prediction.columns.get_loc('Home_2nd_Last_Win')  
    home_3rd_col = prediction.columns.get_loc('Home_3rd_Last_Win')  
    home_4th_col = prediction.columns.get_loc('Home_4th_Last_Win')  
    home_5th_col = prediction.columns.get_loc('Home_5th_Last_Win')   

    # loop that iterates backwards from the current row
    for i in range(index, 0, -1): 
        
        # Check if the previous row has the current Team1 played as home or Team2
        if (prediction.iloc[i-1 , home_team_col] == current_home_team or prediction.iloc[i-1 , away_team_col] == current_home_team): 
            
            # Check if the the current Team1 won the last game
            if (current_home_team == prediction.iloc[i-1 , winner_col]):   
                
                # 1 added to the list if the Team1 has won
                home_team_form.append(1)
            else:
                # 0 added to the list if the Team1 has lost
                home_team_form.append(0)
    
    # Check if the list has 1 or more matches outcome, if so update the outcome of the first last game of the Team1
    if ( len(home_team_form) > 0 ): 
        prediction.iloc[index , home_1st_col] = home_team_form[0]
        
    # Check if the list has 2 or more matches outcome, if so update the outcome of the second last game of the Team1
    if ( len(home_team_form) > 1 ):            
        prediction.iloc[index , home_2nd_col] = home_team_form[1]
        
    # Check if the list has 3 or more matches outcome, if so update the outcome of the third last game of the Team1    
    if ( len(home_team_form) > 2 ):
        prediction.iloc[index , home_3rd_col] = home_team_form[2]
        
    # Check if the list has 4 or more matches outcome, if so update the outcome of the fourth last game of the Team1
    if ( len(home_team_form) > 3 ):
        prediction.iloc[index , home_4th_col] = home_team_form[3]
    
    # Check if the list has 5 or more matches outcome, if so update the outcome of the fifth last game of the Team1
    if ( len(home_team_form) > 4 ):
        prediction.iloc[index , home_5th_col] = home_team_form[4]

    # Reset the list to empty for the next iteration
    home_team_form = []

# Function to update the Team2's results (Won or Lost) of the last 5 games

In [ ]:
# def add_away_team_previous_form():

# Array to store the previous form of the Team2 
away_team_form = []

# The main loop that iterates through each row
for index, row in prediction.iterrows():    

    # Team2 of the current row
    current_away_team = row['team2']
    
    # The column indexes of the home and Team2s and the winner
    home_team_col = prediction.columns.get_loc('team1')
    away_team_col = prediction.columns.get_loc('team1')
    winner_col = prediction.columns.get_loc('winner')

    # The column indexes of the Team2's form (last 5 games) 
    away_1st_col = prediction.columns.get_loc('Away_1st_Last_Win')   
    away_2nd_col = prediction.columns.get_loc('Away_2nd_Last_Win')   
    away_3rd_col = prediction.columns.get_loc('Away_3rd_Last_Win')   
    away_4th_col = prediction.columns.get_loc('Away_4th_Last_Win')   
    away_5th_col = prediction.columns.get_loc('Away_5th_Last_Win')        

    # loop that iterates backwards from the current row
    for i in range(index, 0, -1):  
        
        # Check if the previous row has the current Team2 played as home or Team2
        if (prediction.iloc[i-1 , home_team_col] == current_away_team or prediction.iloc[i-1 , away_team_col] == current_away_team): 
            
            # Check if the the current Team2 won the last game
            if (current_away_team == prediction.iloc[i-1 , winner_col]):
                
                # 1 added to the list if the Team2 has won
                away_team_form.append(1)
            else:
                
                # 0 added to the list if the Team2 has lost
                away_team_form.append(0)

    # Check if the list has 1 or more matches outcome, if so update the outcome of the first last game of the Team2
    if ( len(away_team_form) > 0 ):
        prediction.iloc[index , away_1st_col] = away_team_form[0]
    
    # Check if the list has 2 or more matches outcome, if so update the outcome of the second last game of the Team2
    if ( len(away_team_form) > 1 ):
        prediction.iloc[index , away_2nd_col] = away_team_form[1]
    
    # Check if the list has 3 or more matches outcome, if so update the outcome of the third last game of the Team2
    if ( len(away_team_form) > 2 ):
        prediction.iloc[index , away_3rd_col] = away_team_form[2]
    
    # Check if the list has 4 or more matches outcome, if so update the outcome of the fourth last game of the Team2
    if ( len(away_team_form) > 3 ):
        prediction.iloc[index , away_4th_col] = away_team_form[3]
    
    # Check if the list has 5 or more matches outcome, if so update the outcome of the fifth last game of the Team2
    if ( len(away_team_form) > 4 ):
        prediction.iloc[index , away_5th_col] = away_team_form[4]


    # Reset the list to empty for the next iteration
    away_team_form = []

# Function to update the Team1's results against the Team2 (Won or Lost) of the last 3 encounters

In [ ]:
# def add_home_and_away_team_encounters_result():
     
# Array to store the previous encounters results of the Team1 
home_form_against_away = []

# The main loop that iterates through each row
for index, row in prediction.iterrows():    
    
    # Home and Team2s of the current row
    current_home_team = row['team1'] 
    current_away_team = row['team2'] 

    # The column indexes of the home and Team2s and the winner
    home_team_col = prediction.columns.get_loc('team1')
    away_team_col = prediction.columns.get_loc('team1')
    winner_col = prediction.columns.get_loc('winner')

    # The column indexes of the Team1's result against Team2 (last 3 games) 
    home_encounter_1st_col = prediction.columns.get_loc('Home_Last_Win_Against_Away')
    home_encounter_2nd_col = prediction.columns.get_loc('Home_2nd_Last_Win_Against_Away')
    home_encounter_3rd_col = prediction.columns.get_loc('Home_3rd_Last_Win_Against_Away')

    # loop that iterates backwards from the current row
    for i in range(index, 0, -1):            
        
        did_home_team_played_last_game = False
        did_away_team_played_last_game = False
        
        # Check if the previous row has the current Team1 played as home or Team2
        if ( prediction.iloc[i-1 , home_team_col] == current_home_team or prediction.iloc[i-1 , away_team_col] == current_home_team ):
            did_home_team_played_last_game = True
        
        # Check if the previous row has the current Team2 played as home or Team2
        if ( prediction.iloc[i-1 , home_team_col] == current_away_team or prediction.iloc[i-1 , away_team_col] == current_away_team ):
            did_away_team_played_last_game = True
              
        # Check if both home and Team2s have played
        if (did_home_team_played_last_game and did_away_team_played_last_game ):
            
            # Check if the the current Team1 won the previous encounter against Team2
            if (current_home_team == prediction.iloc[i-1 , winner_col]):                
                home_form_against_away.append(1)                    
            else:
                home_form_against_away.append(0)

    # Check if the list has 1 or more matches outcome, if so update the outcome of the first last encounter
    if ( len(home_form_against_away) > 0 ):
        prediction.iloc[index , home_encounter_1st_col] = home_form_against_away[0]
    
    # Check if the list has 2 or more matches outcome, if so update the outcome of the second last encounter
    if ( len(home_form_against_away) > 1 ):
        prediction.iloc[index , home_encounter_2nd_col] = home_form_against_away[1]
    
    # Check if the list has 3 or more matches outcome, if so update the outcome of the third last encounter
    if ( len(home_form_against_away) > 2 ):
        prediction.iloc[index , home_encounter_3rd_col] = home_form_against_away[2]
    
    # Reset the list to empty for the next iteration
    home_form_against_away = []

In [ ]:
prediction_df

In [ ]:
prediction

# Removing Winner Column

In [ ]:
prediction_df = prediction_df.drop('winner', axis=1)
prediction_df

# Encoding all prediction before train_test_split

In [ ]:
# All data
ohe = OneHotEncoder(sparse=False)
prediction_df_tmp = pd.DataFrame(prediction_df)
categ_obj = [col for col in prediction_df_tmp.columns if prediction_df_tmp[col].dtypes == 'O' ]

prediction_ohe = pd.DataFrame((ohe.fit_transform(prediction_df[categ_obj])), columns = ohe.get_feature_names())
prediction_ohe.index = prediction_df.index
prediction_num = prediction_df.drop(categ_obj, axis=1)

prediction_new = pd.concat([prediction_ohe, prediction_num], axis=1)
prediction_new

# Train/Test Split 

In [ ]:
# This split gives an unchanging result
from sklearn.preprocessing import StandardScaler
X = prediction_new.drop('team1_win', axis=1)
target = prediction_new['team1_win']
target=target.astype(int)

# MAYBE ADD VALID
#Splitting the data into training and testing data and scaling it
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.25, random_state=0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Different Machine Learning Models Testing

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
score_lg = cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X, target, cv=10)
np.average(score_lg)

In [ ]:
score_svc = cross_val_score(SVC(gamma='auto'), X, target, cv=10)
np.average(score_svc)

In [ ]:
score_rf = cross_val_score(RandomForestClassifier(n_estimators=10), X, target, cv=10)
np.average(score_rf)

# Testing MLMs without cross validation

In [ ]:
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

In [ ]:
lr = LogisticRegression(solver='liblinear',multi_class='ovr')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)